In [2]:
from websockets.client import connect
import os
import dotenv

from src.data_ingestion.listen_news import send_new

dotenv.load_dotenv()

auth = {
  "action": "auth",
  "key": os.environ["APCA_API_KEY_ID"],
  "secret": os.environ["APCA_API_SECRET_KEY"]
}
async with connect("wss://paper-api.alpaca.markets/stream", extra_headers=auth) as websocket:
    greeting = await websocket.recv()
    async for message in websocket:
      print(message)
      send_new(message)
    print(f"<<< {greeting}")



## Get last news and send to Kafka topic

In [ ]:
import pandas as pd
import requests
import json 


url = "https://data.alpaca.markets/v1beta1/news"#?symbols=COIN,BTCUSD"

headers = {
    "accept": "application/json",
    "APCA-API-KEY-ID": "PKVUS0LW2MYNR51ID5ZS",
    "APCA-API-SECRET-KEY": "AUuz3zFpHVisDsfhUKeBkMUeUEXOjOXX9BRSUilE"
}

response = requests.get(url, headers=headers)

news_response = json.loads(response.text)

In [ ]:
news_response_df = pd.DataFrame(news_response["news"])
news_response_df

In [ ]:
from src.data_ingestion.listen_news import send_new

for i in news_response_df.to_dict(orient='records'):
    send_new(i)